In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


base_url = "https://www.carwale.com/used/mumbai/"


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}


def scrape_page(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    
    cars = []
    
    
    car_cards = soup.find_all("div", class_="used-car-listing-card")  
    for card in car_cards:
        try:
            
            name = card.find("h3", class_="heading-6").text.strip()  
            price = card.find("span", class_="amount").text.strip()  
            details = card.find_all("li", class_="key-feature") 
            
            mileage = details[0].text.strip() if len(details) > 0 else "N/A"
            fuel_type = details[1].text.strip() if len(details) > 1 else "N/A"
            location = card.find("div", class_="seller-city").text.strip() 
            
           
            cars.append({
                "Name": name,
                "Price": price,
                "Mileage": mileage,
                "Fuel Type": fuel_type,
                "Location": location,
            })
        except Exception as e:
            print(f"Error parsing car card: {e}")
    
    return cars


def scrape_all_pages(base_url, max_pages=5):
    all_cars = []
    for page in range(1, max_pages + 1):
        print(f"Scraping page {page}...")
        url = f"{base_url}?page={page}"  
        cars = scrape_page(url)
        all_cars.extend(cars)
    
    return all_cars


if __name__ == "__main__":
    max_pages_to_scrape = 5  
    car_data = scrape_all_pages(base_url, max_pages_to_scrape)
    
    
    df = pd.DataFrame(car_data)
    df.to_csv("used_cars_mumbai.csv", index=False)
    print("Scraping completed! Data saved to 'used_cars_mumbai.csv'.")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping completed! Data saved to 'used_cars_mumbai.csv'.
